In [ ]:
import os
import csv
import math
import pandas as pd
import numpy as np

#read speach from file
import moviepy.editor as mp 
import speech_recognition as sr 

train_dir = 'data/'

In [ ]:
def set_segment_to_ads(advertisement_ID):
    df_train_segments = pd.read_csv('train_segments.csv', sep=',')
    arrays = df_train_segments.values
    s = 1
    for adsid, segid in arrays.tolist(): 
        if adsid == advertisement_ID:
            s = segid
            break    
    return s;

In [ ]:

def prepared_data_from_audio(filename):
    if filename != '.DS_Store':
        video = mp.VideoFileClip(filename=train_dir+ str(filename))
        audio_file = video.audio
        advertsement_id = filename.split('.', 1)[0]
        advertsement_id = int(advertsement_id)
        file_extention = filename.split('.', 1)[1]

        audio_file_name = str(advertsement_id) +'.wav'
        audio_file.write_audiofile(audio_file_name)

        # Initialize recognizer
        r = sr.Recognizer()
        r.pause_threshold = 10.0 #время ожидания слова
        with sr.AudioFile(audio_file_name) as source:
            audio_text = r.listen(source)
                
        text = ""

        try:
            text = r.recognize_google(audio_text, language = "ru")
        except Exception:
            print()
        # with sr.AudioFile(audio_file_name) as source:
        #     audio_text = r.listen(source)
        #     size_ms = 8000
        #     total_clip_time_round_ms = math.ceil(video.end) * 1000
        #     clip_parts = math.ceil(total_clip_time_round_ms / size_ms)
        #     start_ms = 0
        #     end_ms = 0
        #     part = 0
        #     text = ""

        #     while part < clip_parts:
        #         start_ms = part * size_ms
        #         end_ms = (part + 1) * size_ms
        #         audio_segment = audio_text.get_segment(start_ms, end_ms)
        #         if audio_segment.frame_data:
        #             text_part = r.recognize_google(audio_segment, language = "ru")
        #             text = text + ' ' + text_part
        #         part = part + 1
        segment_id = set_segment_to_ads(advertsement_id)
        row = [[advertsement_id, segment_id, file_extention, video.end, text, None]]
        os.remove(audio_file_name)
        return row;

In [ ]:
train_dataset_file = 'train_dataset.csv'

if not os.path.isfile(train_dataset_file):

    #init train_dataset_file with headers
    headers = ['AdvertisementID', 'SegmentID', 'FileExtention', 'Time', 'SpeechText', 'CadrText']    
    with open(train_dataset_file, mode='w', newline='', encoding='utf-8') as file:
        write = csv.writer(file)
        write.writerow(headers)

    for filename in os.listdir(path=train_dir)[0:]:
        if filename != '.DS_Store':    
            row = prepared_data_from_audio(filename)
            df = pd.DataFrame(row)
            df.to_csv(train_dataset_file, sep=',', index=False, mode='a', header=False)
else:
    last_advertisement_id = 0

    df = pd.read_csv(train_dataset_file)
    last_advertisement_id = str(df.iat[-1, 0])
    last_format = str(df.iat[-1, 2])
    last_name = last_advertisement_id + '.' + last_format
    last_index = os.listdir(path=train_dir).index(last_name) + 1

    for filename in os.listdir(path=train_dir)[last_index:]:
        row = prepared_data_from_audio(filename)
        df = pd.DataFrame(row)
        df.to_csv(train_dataset_file, sep=',', index=False, mode='a', header=False)
